In [1]:

from mmdet.apis import inference_detector
from mmdet.apis import init_detector
from cfg_infer import cfg

import argparse
import mmcv
import glob as glob
import os
from tqdm import tqdm

# Contruct the argument parser.

Default Config:
optimizer = dict(
    type='SGD',
    lr=0.01,
    momentum=0.9,
    weight_decay=0.0005,
    nesterov=True,
    paramwise_cfg=dict(norm_decay_mult=0.0, bias_decay_mult=0.0))
optimizer_config = dict(grad_clip=None)
lr_config = dict(
    policy='YOLOX',
    warmup='exp',
    by_epoch=False,
    warmup_by_epoch=True,
    warmup_ratio=1,
    warmup_iters=5,
    num_last_epochs=15,
    min_lr_ratio=0.05)
runner = dict(type='EpochBasedRunner', max_epochs=300)
checkpoint_config = dict(interval=10)
log_config = dict(interval=50, hooks=[dict(type='TextLoggerHook')])
custom_hooks = [
    dict(type='YOLOXModeSwitchHook', num_last_epochs=15, priority=48),
    dict(type='SyncNormHook', num_last_epochs=15, interval=10, priority=48),
    dict(
        type='ExpMomentumEMAHook',
        resume_from=None,
        momentum=0.0001,
        priority=49)
]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
opencv_num_threads 

In [2]:
args = {}
args['weights'] = 'outputs/yolox_l_8x8_300e_coco/epoch_255.pth'
args['input'] = '/home/ubuntu/mm/input/data_root/dataset/JPEGImages'

In [3]:
import torch

In [5]:
raw = torch.load(args['weights'], map_location='cpu')

In [6]:
torch.save(raw, 'cpu.pth')

In [8]:
print(cfg.device)
model = init_detector(cfg, 'cpu.pth')

cpu
load checkpoint from local path: cpu.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_backbone_stem_conv_conv_weight, ema_backbone_stem_conv_bn_weight, ema_backbone_stem_conv_bn_bias, ema_backbone_stem_conv_bn_running_mean, ema_backbone_stem_conv_bn_running_var, ema_backbone_stem_conv_bn_num_batches_tracked, ema_backbone_stage1_0_conv_weight, ema_backbone_stage1_0_bn_weight, ema_backbone_stage1_0_bn_bias, ema_backbone_stage1_0_bn_running_mean, ema_backbone_stage1_0_bn_running_var, ema_backbone_stage1_0_bn_num_batches_tracked, ema_backbone_stage1_1_main_conv_conv_weight, ema_backbone_stage1_1_main_conv_bn_weight, ema_backbone_stage1_1_main_conv_bn_bias, ema_backbone_stage1_1_main_conv_bn_running_mean, ema_backbone_stage1_1_main_conv_bn_running_var, ema_backbone_stage1_1_main_conv_bn_num_batches_tracked, ema_backbone_stage1_1_short_conv_conv_weight, ema_backbone_stage1_1_short_conv_bn_weight, ema_backbone_stage1_1_short_conv_bn_bias, 

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
image_paths = glob.glob(f"{args['input']}/*.jpg")

with open('input/data_root/dataset/ImageSets/Main/train.txt') as f:
    files = f.readlines()

for i, file in enumerate(tqdm(files)):
    image_path = 'input/data_root/dataset/JPEGImages/{}.jpg'.format(file[:-1])

    image = mmcv.imread(image_path)
    # Carry out the inference.
    result = inference_detector(model, image)
    print(result)
    # Show the results.
    frame = model.show_result(image, result, score_thr=args['threshold'])
    # mmcv.imshow(frame)
    # Initialize a file name to save the reuslt.
    save_name = f"{image_path.split(os.path.sep)[-1].split('.')[0]}"
    mmcv.imwrite(frame, f"pred/{i}.jpg")
    print(f"pred/{i}.jpg")